
# An Introduction to Redis with Python

[Redis Data Types introduction](http://redis.io/topics/data-types-intro) managed by [redis-py](https://github.com/andymccurdy/redis-py) Python client

In [ ]:
pip install redis

In [ ]:
import redis

In [ ]:
r = redis.StrictRedis(host='localhost', port=6379, db=0)

## Getting and settings Redis [*Keys*](http://redis.io/topics/data-types-intro#redis-keys)

In [ ]:
r.set('my.key', 'value1')

In [ ]:
r.get('my.key')

We can also check for the existence of a given key.

In [ ]:
r.exists('my.key')

In [ ]:
r.exists('some.other.key')

If we want to set multiple keys at once, we can use [MSET](http://redis.io/commands/mset) and pass a Python dictionary as follows.

In [ ]:
r.mset({'my.key':'value2', 'some.other.key':123})

In [ ]:
r.get('my.key')

In [ ]:
r.get('some.other.key')

We can also increment the value of a given key in an atomic way.

In [ ]:
r.incrby('some.other.key',10)

Notice how the resulting type has been changed to integer!

### Setting keys to expire

With **redis-py** we can also set keys with limited time to live.

In [ ]:
r.expire('some.other.key',1)
r.exists('some.other.key')

Let's wait for a couple of seconds for the key to expire and check again.

In [ ]:
from time import sleep
sleep(2)
r.exists('some.other.key')

Finally, **del** is a reserved keyword in the Python syntax. Therefore redis-py uses 'delete' instead.

In [ ]:
r.delete('my.key')

## Redis [*Lists*](http://redis.io/topics/data-types-intro#redis-lists)

Redis lists are linked lists of keys. We can insert and remove elements from both ends.

The [LPUSH](http://redis.io/commands/lpush) command adds a new element into a list, on the left.

In [ ]:
r.lpush('my.list', 'elem1')

The [RPUSH](http://redis.io/commands/rpush) command adds a new element into a list, on the right. 

In [ ]:
r.rpush('my.list', 'elem2')

Finally the [LRANGE](http://redis.io/commands/lrange) command extracts ranges of elements from lists.  

In [ ]:
r.lrange('my.list',0,-1)

In [ ]:
r.lpush('my.list', 'elem0')

In [ ]:
r.lrange('my.list',0,-1)

The result is returned as a Python list. We can use [LLEN](http://redis.io/commands/llen) to check a Redis list lenght without requiring to store the result of `lrange` and then use Python's `len`.

In [ ]:
r.llen('my.list')

We can push multiple elements with a single call to push.

In [ ]:
r.rpush('my.list','elem3','elem4')

In [ ]:
r.lrange('my.list',0,-1)

Finally, we have the equivalent pop operations for both, right and left ends.

In [ ]:
r.lpop('my.list')

In [ ]:
r.lrange('my.list',0,-1)

In [ ]:
r.rpop('my.list')

In [ ]:
r.lrange('my.list',0,-1)

### Capped Lists

We can also [TRIM](http://redis.io/commands/ltrim) Redis lists with redis-py. We need to pass three arguments: the name of the list, and the start and stop indexes.

In [ ]:
r.lpush('my.list','elem0')
r.ltrim('my.list',0,2)

In [ ]:
r.lrange('my.list',0,-1)

Notice as the last element has been dropped when triming the list. The `lpush`/`ltrim` sequence is a common pattern when inserting in a list that we want to keep size-fized.

In [ ]:
r.delete('my.list')

## Redis [*Hashes*](http://redis.io/topics/data-types-intro#redis-hashes)  

The equivalent of Python dictionaries are Redis *hashes*, with field-value pairs. We use the command [HMSET](http://redis.io/commands/hmset).

In [ ]:
r.hmset('my.hash', {'field1':'value1',
                   'field2': 1234})

We can also set individual fields.

In [ ]:
r.hset('my.hash','field3',0)

We have methods to get individual and multiple fields from a hash.

In [ ]:
r.hget('my.hash','field2')

In [ ]:
r.hmget('my.hash','field1','field2','field3')

The result is returned as a list of values.

Increment operations are also available for hash fields.

In [ ]:
r.hincrby('my.hash','field2',10)

## Redis [Sets](http://redis.io/topics/data-types-intro#redis-sets)

Redis Sets are unordered collections of strings. We can easily add multiple elements to a Redis set in redis-py as follows by using its implementation of [SADD](http://redis.io/commands/sadd).

In [ ]:
r.sadd('my.set', 1, 2, 3)

As a result, we get the size of the set. If we want to check the elements within a set, we can use [SMEMBERS]().

In [ ]:
r.smembers('my.set')

In [ ]:
type(r.smembers('my.set'))

Notice that we get a Python *set* as a result. That opens the door to all sort of Python set operations. However, we can operate directly within the Redis server space, and still do things as checking an element membership using [SISMEMBER](http://redis.io/commands/sismember). This is the way to go for enterprise applications in order to keep our system **scalability** and **availability** (i.e. large sets, concurrent access, etc).

In [ ]:
r.sismember('my.set', 4)

In [ ]:
r.sismember('my.set', 1)

The [SPOP](http://redis.io/commands/spop) command extracts a random element (and we can use [SRANDMEMBER](http://redis.io/commands/srandmember) to get one or more random elements without extraction).

In [ ]:
elem = r.spop('my.set')

In [ ]:
r.smembers('my.set')

In [ ]:
r.sadd('my.set',elem)

In [ ]:
r.smembers('my.set')

Or if we want to be specific, we can just use [SREM](http://redis.io/commands/srem).

In [ ]:
r.srem('my.set',2)

In [ ]:
r.smembers('my.set')

### Set operations

In order to obtain the intersection between two sets, we can use [SINTER](http://redis.io/commands/sinter).

In [ ]:
r.sadd('my.other.set', 'A','B',1)

In [ ]:
r.smembers('my.other.set')

In [ ]:
r.sinter('my.set','my.other.set')

That we get as a Python set. Alternatively, we can directly store the result as a new Redis set by using [SINTERSTORE](http://redis.io/commands/sinterstore).

In [ ]:
r.sinterstore('my.intersection','my.set','my.other.set')

In [ ]:
r.smembers('my.intersection')

Similar operations are available for union and difference. Moreover, they can be applied to more than two sets. For example, let's create a union set with all the previous and store it in a new Redis set.

In [ ]:
r.sadd('my.intersection','batman')
r.sunionstore('my.union','my.set','my.other.set','my.intersection')

In [ ]:
r.smembers('my.union')

Finally, the number of elements of a given Redis set can be obtained with [SCARD](http://redis.io/commands/scard).

In [ ]:
r.scard('my.union')

Let's clean our server before leaving this section.

In [ ]:
r.delete('my.set','my.other.set','my.intersection','my.union')

In [ ]:
r.zrank('my.sorted.set','third')